In [1]:
import pandas as pd
import numpy as np

##### 偏相関係数

#### データについて
```
前回用いたサンプルデータは、基本的にはa1-a7が0or1のみの二値で構成される事実上のカテゴリカルデータ*1で、cvも"Yes"or"No"のみの二値で構成されるカテゴリカルデータです。
記事とデータは以下の記事から取得しました。
```
https://tjo.hatenablog.com/entry/2013/08/15/001338
https://github.com/ozt-ca/tjo.hatenablog.samples/blob/master/r_samples/public_lib/jp/conflict_sample.txt

In [21]:
cv_data = pd.read_csv("./data/Partial_correlation.csv")
cv_data.head()

,a1,a2,a3,a4,a5,a6,a7,cv
0,0,1,0,1,0,0,0,No
1,0,0,1,1,0,0,1,No
2,1,1,0,1,0,0,0,No
3,0,1,1,1,0,0,1,No
4,1,1,1,1,0,0,0,No


In [22]:
def conv_dummy(x):
    if x == "No":
        return 0
    else:
        return 1

In [23]:
cv_data["cv"] = cv_data["cv"].apply(lambda x : conv_dummy(x))
cv_data.head()

,a1,a2,a3,a4,a5,a6,a7,cv
0,0,1,0,1,0,0,0,0
1,0,0,1,1,0,0,1,0
2,1,1,0,1,0,0,0,0
3,0,1,1,1,0,0,1,0
4,1,1,1,1,0,0,0,0


#### 説明変数が３変数以上の偏相関係数の算出方法
```
1.相関行列の逆行列を求める
2.逆行列の各要素を2つの対角要素の積の平方根で割り，符号を逆転する
```
https://www.fujiitoshiki.com/improvesociety/?p=4026

In [24]:
# 偏相関係数
def p_corr(y,data):
    corr = data.corr()
    #1.相関行列の逆行列を求める
    inv_corr = pd.DataFrame(np.linalg.inv(corr),columns=corr.columns,index=corr.columns)
    p_corr_list = []
    #2.逆行列の各要素を2つの対角要素の積の平方根で割り，符号を逆転する
    for x in corr.columns:
        p_corr = -(inv_corr[x][y] / np.sqrt(inv_corr[x][x]*inv_corr[y][y]))
        p_corr_list.append([x ,p_corr])
    return p_corr_list

In [25]:
y =  "cv"
cv_data = pd.read_csv("./data/Partial_correlation.csv")
cv_data["cv"] = cv_data["cv"].apply(lambda x : conv_dummy(x))
cv_data=p_corr(y, cv_data)
cv_data

[['a1', 0.11164748366062084],
 ['a2', -0.05897922291852616],
 ['a3', 0.0029857354080831643],
 ['a4', -0.2840844729564049],
 ['a5', 0.17556247482163043],
 ['a6', 0.8072447825054478],
 ['a7', 0.005953910517755407],
 ['cv', -1.0]]

#### 検証
```
算出結果が正しいか検証する。こちらの記事で同じデータを活用して
偏相関係数を算出していたので検証結果が正しいか確認してみます。

```
https://blog.goo.ne.jp/r-de-r/e/0e9a855c5f2d536a6b74e05a26f5a27d

```
 a1     a2     a3     a4     a5     a6     a7     cv
a1  1.000 -0.060 -0.003 -0.115  0.101  0.163  0.006  0.203
a2 -0.060  1.000  0.002  0.101 -0.070 -0.151 -0.027 -0.166
a3 -0.003  0.002  1.000  0.007 -0.018  0.022 -0.031  0.015
a4 -0.115  0.101  0.007  1.000 -0.196 -0.484  0.009 -0.547
a5  0.101 -0.070 -0.018 -0.196  1.000  0.335 -0.004  0.377
a6  0.163 -0.151  0.022 -0.484  0.335  1.000 -0.011  0.871
a7  0.006 -0.027 -0.031  0.009 -0.004 -0.011  1.000 -0.007
cv  0.203 -0.166  0.015 -0.547  0.377  0.871 -0.007  1.000
```

```
偏相関係数の値が合致することを確かめられた。
```